In [11]:
# libraries
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader

# custom
import model
import utilities
from datasets.dataset_IMADS import DatasetTrain, DatasetTest, DatasetFromNumpy

In [12]:
MACHINE = 'BrushlessMotor' # choose between BrushlessMotor and RoboticArm
WINDOW_SIZE_MS = 100 # milliseconds

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

PARAMS = {
    'layer_dims': [2048, 2048, 2048, 16],
    'lr': 0.0001,
    'criterion': utilities.MSE,
    'batch_size': 1024,
    'num_epochs': 1000,
    'patience': 3,
    'normalisation': 'std_window', # ('std', 'min-max', 'std_window', or 'min-max_window')
    'valid_size': 0.1,
    'seed': 1995,
    'device':device
}

# Set the seed for general torch operations
torch.manual_seed(PARAMS['seed'])
# Set the seed for MPS torch operations (ones that happen on the MPS Apple GPU)

if device == 'mps':
    torch.mps.manual_seed(PARAMS['seed'])
elif device == 'cuda':
    torch.cuda.manual_seed(PARAMS['seed'])
elif device == 'cpu':
    torch.manual_seed(PARAMS['seed'])
else:
    raise ValueError(f"Wrong device value: {device}")

In [ ]:
train_dataset = DatasetTrain(machine=MACHINE, window_size_ms=WINDOW_SIZE_MS, params=PARAMS)
X_valid, y_valid = train_dataset.get_valid_dataset()
valid_dataset = DatasetFromNumpy(X_valid, y_valid['anomaly_label'].to_list(), device, PARAMS['normalisation'])
test_dataset = DatasetTest(machine=MACHINE, window_size_ms=WINDOW_SIZE_MS, params=PARAMS)

train_data_loader = DataLoader(
    train_dataset, batch_size=PARAMS['batch_size'], shuffle=True)
valid_data_loader = DataLoader(
    valid_dataset, batch_size=PARAMS['batch_size'], shuffle=False)
test_data_loader = DataLoader(
    test_dataset, batch_size=PARAMS['batch_size'], shuffle=False)

In [15]:
# Extract the number of channels and window lengths for each sensor\n",
NUM_CHANNELS = [x.shape[1] for x in train_dataset.X]
WINDOW_LENGTHS = [x.shape[2] for x in train_dataset.X]
SENSORS = train_dataset.sensor_dict

Train

In [ ]:
baseline = model.AutoencoderFC(WINDOW_LENGTHS, NUM_CHANNELS, PARAMS, SENSORS)
optimizer = torch.optim.Adam(baseline.parameters(), lr=PARAMS['lr'])
baseline.fit(train_data_loader, valid_data_loader, optimizer)

In [ ]:
AUC_scores = baseline.test(test_data_loader, test_dataset.y, utilities.MSE, 'median')


## Show results table as in paper

In [ ]:
results = AUC_scores.copy()
results.columns = ['S+T', 'Source', 'Target']
new_order = [
    'total_loss',
    'f_ism330dhcx_acc',
    's_ism330dhcx_acc',
    'f_ism330dhcx_gyro',
    's_ism330dhcx_gyro',
    'f_imp23absu_mic',
    's_imp23absu_mic']
results = results.reindex(new_order)
results.index = [
    'Overall',
    'F-acc',
    'S-acc',
    'F-gyr',
    'S-gyr',
    'F-mic',
    'S-mic']
results = results * 100
results = results.round(2)
results


## Save model and results

In [ ]:
import os
MODEL_PATH = f'models/{MACHINE}'
os.makedirs(MODEL_PATH, exist_ok=True)

torch.save(
    baseline.state_dict(),
    MODEL_PATH +
    os.sep +
    f'baseline_seed{PARAMS["seed"]}.pth')


In [ ]:
RESULTS_PATH = f'results/{MACHINE}'
os.makedirs(RESULTS_PATH, exist_ok=True)
results.to_csv(RESULTS_PATH + os.sep + 'AUC_scores.csv')
